<a href="https://colab.research.google.com/github/MichiganDataScienceTeam/W26-Uber-Data-Analysis/blob/main/Week_2_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import the relevant libraries.

In [9]:
import requests
import re
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
from urllib.parse import quote

2. Load the "corrupted" dataset. Make sure to run methods like .head() and .info() to make sure it was loaded properly!

In [10]:
df = pd.read_csv(
    "UberDataset [CORRUPTED].csv")

3. Iterate through all dataframe columns to check for NaN (empty) cells. If you find one, override the cell with '**NO_DATA**'.

In [11]:
for col in df.columns:
    df[col] = df[col].fillna("NO_DATA")

4. Focus at the START and STOP columns. Scroll around a bit and find the typos. Make a custom pattern that can be used for both typo formats. **OVERRIDE** the previous cell.

The correct words should be: Rawalpindi and Karachi.

Hint: use r'STRING' (raw) for your pattern, and use .findall(pattern, string).

In [12]:
pattern = r'([A-Z][a-z]*)\?([a-z]*)'
for i in df.index:
  startVal = df.at[i, 'START']
  stopVal = df.at[i, 'STOP']
  if '?' in startVal:
    grouping = re.findall(pattern, startVal)[0]
    firstPart = grouping[0]
    secondPart = grouping[1]
    fixedWord = f'{firstPart}a{secondPart}'
    df.at[i, 'START'] = fixedWord


  if '?' in stopVal:
    grouping = re.findall(pattern, stopVal)[0]
    firstPart = grouping[0]
    secondPart = grouping[1]
    fixedWord = f'{firstPart}a{secondPart}'
    df.at[i, 'STOP'] = fixedWord

df.head(500)

,START_DATE,END_DATE,CATEGORY,START,STOP,MILES,PURPOSE
0,11-05-2016 19:20,11-05-2016 19:28,Business,Agnew,Agnew,2.2,NO_DATA
1,11-04-2016 22:12,11-04-2016 22:25,Business,Cory,Agnew,3.9,NO_DATA
2,11-05-2016 17:29,11-05-2016 17:40,Business,Renaissance,Agnew,2.8,NO_DATA
3,11-06-2016 16:05,11-06-2016 16:22,Business,Renaissance,Agnew,2.8,Meeting
4,3/17/2016 17:20,3/17/2016 18:02,Business,Midtown,Alief,15.5,Meal/Entertain
...,...,...,...,...,...,...,...
495,12/28/2016 18:33,12/28/2016 18:56,Business,Karachi,Karachi,3.8,Errand/Supplies
496,12/28/2016 22:44,12/28/2016 23:18,Business,Karachi,Karachi,5.1,Errand/Supplies
497,12/29/2016 0:49,12/29/2016 1:06,Business,Karachi,Karachi,3.8,Errand/Supplies
498,12/29/2016 12:25,12/29/2016 12:33,Business,Karachi,Karachi,1.4,Errand/Supplies


5. Override all 'Unknown Location' values to 'NO_DATA'




In [13]:
for i in df.index:
  startVal = df.at[i, 'START']
  stopVal = df.at[i, 'STOP']
  if 'Unknown' in startVal:
    df.at[i, 'START'] = 'NO_DATA'
  if 'Unknown' in stopVal:
    df.at[i, 'STOP'] = 'NO_DATA'
df.tail(500)

,START_DATE,END_DATE,CATEGORY,START,STOP,MILES,PURPOSE
654,07-03-2016 00:28,07-03-2016 00:38,Business,Cary,Morrisville,3.1,Errand/Supplies
655,07-06-2016 20:04,07-06-2016 20:14,Business,Cary,Morrisville,3.3,Meal/Entertain
656,07-07-2016 08:22,07-07-2016 08:50,Business,Cary,Morrisville,7.9,Temporary Site
657,7/13/2016 12:39,7/13/2016 13:20,Personal,Cary,Morrisville,23.5,NO_DATA
658,7/14/2016 15:51,7/14/2016 15:59,Personal,Cary,Morrisville,3.3,NO_DATA
...,...,...,...,...,...,...,...
1149,11-01-2016 19:14,11-01-2016 19:20,Business,Whitebridge,Whitebridge,1.0,NO_DATA
1150,11-01-2016 19:59,11-01-2016 20:12,Business,Whitebridge,Whitebridge,4.1,NO_DATA
1151,11-01-2016 20:41,11-01-2016 20:55,Business,Whitebridge,Whitebridge,4.2,Meal/Entertain
1152,02-02-2016 13:04,02-02-2016 13:23,Business,Whitebridge,Williamsburg Manor,8.3,Meeting


6. Convert all the START_DATE and END_DATE values to datetime objects.

In [14]:
df['START_DATE'] = pd.to_datetime(df['START_DATE'], errors='coerce', format='mixed')
df['END_DATE'] = pd.to_datetime(df['END_DATE'], errors='coerce', format='mixed')

7. Create a function that validate that timings for the trip makes sense (start date should be less than end date), and if not, sets both START_DATE and END_DATE to NaT



In [15]:
def invalid_trip_time(df: pd.DataFrame) -> pd.DataFrame:
  invalid = df["START_DATE"] >= df["END_DATE"]

  df.loc[invalid, "START_DATE"] = pd.NaT
  df.loc[invalid, "END_DATE"] = pd.NaT

  return df

8. Extract these specific time features: hour, day of the week, if it is rush hour (let rush hour be between 7-9 am and 4-6 pm), and if it is the weekend (based on START_DATE column)

In [16]:
df["hour"] = df["START_DATE"].dt.hour
df["day_of_week"] = df["START_DATE"].dt.dayofweek
df["is_weekend"] = df["day_of_week"] > 5
df["is_rush_hour"] = ((df["hour"].between(7,9)) |  (df["hour"].between(16,18)))

# New Section

9. Connect to our **GitHub**. The API data is divided up into hourly segments. Using both the **START** location and the **START_DATE**, as well as the datetime attributes, make a web request for each of your rows.

Hint：Find & get the corresponding index in the hourly list, then use that to get the other attributes.

PLEASE CACHE YOUR DATA TO AVOID BURNING THE SERVER.



In [17]:
# 1. Create a local directory for the cache
cache_dir = 'cached'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

for idx, i in enumerate(df['START']):
    # The local filename where we store the JSON
    cache_file = os.path.join(cache_dir, f"{i}.json")

    # 2. CHECK CACHE: Only make a request if the file isn't local
    if os.path.exists(cache_file):
        with open(cache_file, 'r') as f:
            firstRequest = json.load(f)['hourly']
    else:
        url = f'https://23likenny.github.io/MDSTUber/Week2/{i}.json'
        holder = requests.get(url)

        if holder.status_code == 200:
            full_json = holder.json()
            # SAVE to cache for next time
            with open(cache_file, 'w') as f:
                json.dump(full_json, f)
            firstRequest = full_json['hourly']
        else:
            print(f"Skipping Index {idx}: {i} not found at URL.")
            continue

    x = firstRequest['time']
    currentTime = df.at[idx, 'START_DATE']
    target_str = currentTime.strftime('%Y-%m-%dT%H:00')
    find_idx = x.index(target_str)

    # Pulling your variables
    temperature = firstRequest['apparent_temperature'][find_idx]
    is_day = firstRequest['is_day'][find_idx]
    rain = firstRequest['rain'][find_idx]
    snowfall = firstRequest['snowfall'][find_idx]
    showers = firstRequest['showers'][find_idx]
    weather_code = firstRequest['weather_code'][find_idx]
    wind_speed_10m = firstRequest['wind_speed_10m'][find_idx]
    wind_gusts_10m = firstRequest['wind_gusts_10m'][find_idx]
    print(f'Current Index: {idx}. The temperature is {temperature}')


    # Writing to DataFrame
    df.at[idx, 'TEMPERATURE'] = temperature
    df.at[idx, 'IS_DAY'] = is_day
    df.at[idx, 'RAIN'] = rain
    df.at[idx, 'SNOWFALL'] = snowfall
    df.at[idx, 'SHOWERS'] = showers
    df.at[idx, 'WEATHER_CODE'] = weather_code
    df.at[idx, 'WIND_SPEED_10M'] = wind_speed_10m
    df.at[idx, 'WIND_GUSTS_10M'] = wind_gusts_10m

Current Index: 0. The temperature is 49.3
Current Index: 1. The temperature is 41.6
Current Index: 2. The temperature is 52.5
Current Index: 3. The temperature is 43.9
Current Index: 4. The temperature is 50.9
Current Index: 5. The temperature is 75.6
Current Index: 6. The temperature is 91.5
Current Index: 7. The temperature is 73.5
Current Index: 8. The temperature is 35.3
Current Index: 9. The temperature is 44.0
Current Index: 10. The temperature is 30.9
Current Index: 11. The temperature is 55.2
Current Index: 12. The temperature is 76.6
Current Index: 13. The temperature is 96.2
Current Index: 14. The temperature is 91.8
Current Index: 15. The temperature is 73.5
Current Index: 16. The temperature is 60.0
Current Index: 17. The temperature is 44.3
Current Index: 18. The temperature is 43.7
Current Index: 19. The temperature is 45.1
Current Index: 20. The temperature is 28.8
Current Index: 21. The temperature is 51.0
Current Index: 22. The temperature is 52.4
Current Index: 23. Th

In [ ]:
df.head(500)

,START_DATE,END_DATE,CATEGORY,START,STOP,MILES,PURPOSE,TEMPERATURE,IS_DAY,RAIN,SNOWFALL,SHOWERS,WEATHER_CODE,WIND_SPEED_10M,WIND_GUSTS_10M
0,2016-11-05 19:20:00,2016-11-05 19:28:00,Business,Agnew,Agnew,2.2,NO_DATA,49.3,0.0,0.0,0.0,0.0,1.0,4.0,5.0
1,2016-11-04 22:12:00,2016-11-04 22:25:00,Business,Cory,Agnew,3.9,NO_DATA,41.6,0.0,0.0,0.0,0.0,0.0,6.2,13.3
2,2016-11-05 17:29:00,2016-11-05 17:40:00,Business,Renaissance,Agnew,2.8,NO_DATA,52.5,0.0,0.0,0.0,0.0,1.0,7.0,14.8
3,2016-11-06 16:05:00,2016-11-06 16:22:00,Business,Renaissance,Agnew,2.8,Meeting,43.9,1.0,0.0,0.0,0.0,0.0,16.3,34.6
4,2016-03-17 17:20:00,2016-03-17 18:02:00,Business,Midtown,Alief,15.5,Meal/Entertain,50.9,1.0,0.0,0.0,0.0,3.0,6.8,28.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2016-12-28 18:33:00,2016-12-28 18:56:00,Business,Karachi,Karachi,3.8,Errand/Supplies,76.0,0.0,0.0,0.0,0.0,0.0,10.2,19.1
496,2016-12-28 22:44:00,2016-12-28 23:18:00,Business,Karachi,Karachi,5.1,Errand/Supplies,72.5,0.0,0.0,0.0,0.0,0.0,8.1,11.5
497,2016-12-29 00:49:00,2016-12-29 01:06:00,Business,Karachi,Karachi,3.8,Errand/Supplies,70.0,0.0,0.0,0.0,0.0,1.0,8.1,11.2
498,2016-12-29 12:25:00,2016-12-29 12:33:00,Business,Karachi,Karachi,1.4,Errand/Supplies,79.5,1.0,0.0,0.0,0.0,0.0,9.4,20.5


10. Prep the DataFrame so that it would be ready for modeling (don't worry, we won't start modeling just yet!):

a) keep only numeric columns (in a copy)

b) fill in Na values with the group-wise mean for each START location if applicable

c) create these interaction features:
  - Temperature x Wind Speed (Low temp AND high winds = awful combo, high temp AND high winds = good combo)
  - Overall Weather Stress (Hint: summation)

In [22]:
number_cols = df.select_dtypes(include=[np.number]).columns
df_number_only = df[number_cols].copy()

weather_cols = ['TEMPERATURE', 'RAIN', 'WIND_SPEED_10M', 'SNOWFALL', 'SHOWERS', 'WEATHER_CODE', 'WIND_GUSTS_10M']

for col in weather_cols:
  if col in df_number_only.columns:
    # Only fill NaN values in the relevant weather columns
    # Group by 'START' from the original df to get the correct grouping for the mean
    group_mean = df_number_only.groupby(df['START'])[col].transform('mean')
    df_number_only[col] = df_number_only[col].fillna(group_mean)

df_number_only['TEMP_WIND'] = df_number_only['TEMPERATURE'] * df_number_only['WIND_SPEED_10M']
df_number_only['WEATHER_STRESS'] = df_number_only[weather_cols].sum(axis=1)